In [ ]:
!pip install selenium
!apt-get update
!apt install -y chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 476.0/476.0 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.7 MB/s eta 0:00:00
Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Ign:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.c

In [ ]:
from selenium.webdriver import Remote, ChromeOptions
from selenium.webdriver.chromium.remote_connection import ChromiumRemoteConnection
from selenium.webdriver.common.by import By

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup
import requests
import warnings
import time
import json

from google.colab import files

지역명, 체크인 날짜, 체크아웃 날짜, 인원 수를 입력하면 그것에 맞는 호텔 정보를 가져올 수 있게 함.

쿠팡 영상으로 크롤링을 먼저 공부한 후 건우님이 주신 코드를 참고하여 크롤링하려 했으나, JSON을 다루는 것이 익숙하지 않아 내부 코드를 수정하는 것이 어려웠음.

그래서 쿠팡 영상에서 배운 내용을 바탕으로 beautifulsoup으로 각 상품에 접속하는 링크를 리스트에 모은 후 각각의 링크에 들어가서 정보들을 수집하는 방법이나, selenium으로 직접 링크들을 클릭하고 들어가서 정보들을 수집할 수 있는 방법을 적용해 보려했으나 과정에서 계속 오류가 발생함.

다음주까지 다시 공부하고 찾아보며 오류를 해결해 나갈 예정.

In [59]:
# 경고 문구 제거
warnings.filterwarnings("ignore", category=requests.packages.urllib3.exceptions.InsecureRequestWarning)

# bright data
AUTH = 'brd-customer-hl_6b1e4e16-zone-scraping_browser1:57gtuhxt7jou'
SBR_WEBDRIVER = f'https://{AUTH}@zproxy.lum-superproxy.io:9515'

# 총 검색 호텔 개수 및 총 검색 페이지 수 계산
def get_total_page_num(region, start_date, end_date, person):
  url = f"https://www.yeogi.com/domestic-accommodations?searchType=KEYWORD&keyword={region}&autoKeyword=&checkIn={start_date}&checkOut={end_date}&personal={person}&freeForm=false&page=1"
  print(url)
  print('Connecting to Scraping Browser...')
  sbr_connection = ChromiumRemoteConnection(SBR_WEBDRIVER, 'goog', 'chrome')

  with Remote(sbr_connection, options=ChromeOptions()) as driver:
      print('Connected! Navigating...')
      driver.get(url)
      html = driver.page_source
      soup = BeautifulSoup(html, "html.parser")

      try:
         WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "header.css-1psit91 h1")))
         h1_tag = driver.find_element(By.CSS_SELECTOR, "header.css-1psit91 h1")
         total_result_text = h1_tag.text.strip()
         total_result_num = "".join(char for char in total_result_text if char.isdigit())
         print("총 검색 호텔 개수:", total_result_num)
         return int(total_result_num) // 20 + 1

      except Exception as e:
         print(f"Error in get_total_page_num: {e}")
         return 0

link_list = []

def Crawling(region, start_date, end_date, person, total_page_num):
    hotel_data = []

    for page_num in range(1, 3): # 'total_page_num + 1'이 맞지만 시간이 오래 소요되어 일단 3으로 설정함
        url = f"https://www.yeogi.com/domestic-accommodations?searchType=KEYWORD&keyword={region}&autoKeyword=&checkIn={start_date}&checkOut={end_date}&personal={person}&freeForm=false&page={page_num}"
        print('Connecting to Scraping Browser...')
        sbr_connection = ChromiumRemoteConnection(SBR_WEBDRIVER, 'goog', 'chrome')

        with Remote(sbr_connection, options=ChromeOptions()) as driver:
             print('Connected! Navigating...')
             driver.get(url)
             html = driver.page_source
             soup = BeautifulSoup(html, "html.parser")
             items = soup.select("[class=gc-thumbnail-type-seller-card.css-vbesvf]")
             print(len(items))

             # 상품 접속 링크 가져오기
             for item in items:
                 link = f"https://www.yeogi.com{item.get('href')}"
                 print(link)

'''
             time.sleep(5)
             links = driver.find_elements(By.CSS_SELECTOR, "a.gc-thumbnail-type-seller-card")
             for link in links:
                href = link.get_attribute("href")
                driver.get(href)
'''

if __name__ == "__main__":
    region = input("지역명: ")
    start_date = input("체크인 날짜(0000-00-00): ")
    end_date = input("체크아웃 날짜(0000-00-00): ")
    person = input("인원수: ")
    total_page_num = get_total_page_num(region, start_date, end_date, person)
    print(f"총 검색 페이지 개수: {total_page_num}")
    if total_page_num > 0:
        Crawling(region, start_date, end_date, person, total_page_num)

지역명: 서울
체크인 날짜(0000-00-00): 2024-08-17
체크아웃 날짜(0000-00-00): 2024-08-18
인원수: 2
https://www.yeogi.com/domestic-accommodations?searchType=KEYWORD&keyword=서울&autoKeyword=&checkIn=2024-08-17&checkOut=2024-08-18&personal=2&freeForm=false&page=1
Connecting to Scraping Browser...
Connected! Navigating...
총 검색 호텔 개수: 1164
총 검색 페이지 개수: 59
Connecting to Scraping Browser...
Connected! Navigating...


KeyboardInterrupt: 

In [53]:
# 경고 문구 제거
warnings.filterwarnings("ignore", category=requests.packages.urllib3.exceptions.InsecureRequestWarning)

# bright data
AUTH = 'brd-customer-hl_6b1e4e16-zone-scraping_browser1:57gtuhxt7jou'
SBR_WEBDRIVER = f'https://{AUTH}@zproxy.lum-superproxy.io:9515'

# 총 검색 호텔 개수 및 총 검색 페이지 수 계산
def get_total_page_num(region, start_date, end_date, person):
  url = f"https://www.yeogi.com/domestic-accommodations?searchType=KEYWORD&keyword={region}&autoKeyword=&checkIn={start_date}&checkOut={end_date}&personal={person}&freeForm=false&page=1"
  print(url)
  print('Connecting to Scraping Browser...')
  sbr_connection = ChromiumRemoteConnection(SBR_WEBDRIVER, 'goog', 'chrome')
  with Remote(sbr_connection, options=ChromeOptions()) as driver:
      print('Connected! Navigating...')
      driver.get(url)
      html = driver.page_source
      soup = BeautifulSoup(html, "html.parser")

      try:
         WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "header.css-1psit91 h1")))
         h1_tag = driver.find_element(By.CSS_SELECTOR, "header.css-1psit91 h1")
         total_result_text = h1_tag.text.strip()
         total_result_num = "".join(char for char in total_result_text if char.isdigit())
         print("총 검색 호텔 개수:", total_result_num)
         return int(total_result_num) // 20 + 1
      except Exception as e:
         print(f"Error in get_total_page_num: {e}")
         return 0

def Crawling(region, start_date, end_date, person, total_page_num):
    hotel_data = []
    for page_num in range(1, 3): # 'total_page_num + 1'이 맞지만 시간이 오래 소요되어 일단 3으로 설정함
        url = f"https://www.yeogi.com/domestic-accommodations?searchType=KEYWORD&keyword={region}&autoKeyword=&checkIn={start_date}&checkOut={end_date}&personal={person}&freeForm=false&page={page_num}"
        print('Connecting to Scraping Browser...')
        sbr_connection = ChromiumRemoteConnection(SBR_WEBDRIVER, 'goog', 'chrome')
        with Remote(sbr_connection, options=ChromeOptions()) as driver:
             print('Connected! Navigating...')
             driver.get(url)
             html = driver.page_source
             soup = BeautifulSoup(html, "html.parser")

             # 페이지 로딩 대기
             try:
                  WebDriverWait(driver, 10).until(
                      EC.presence_of_element_located((By.XPATH, '//script[@type="application/ld+json"]'))
                  )
                  script_tag = driver.find_element(By.XPATH, '//script[@type="application/ld+json"]')
                  json_data = script_tag.get_attribute('innerText')

                  # JSON 데이터 파싱
                  data = json.loads(json_data)

                  # 호텔/모텔 정보 추출
                  accommodations = data['mainEntity']['itemListElement']
                  for accommodation in accommodations:
                      hotel_info = {}
                      item = accommodation['item']
                      name = item['name']
                      url = item['url']
                      star_rating = item['starRating']
                      price_range = item['priceRange']
                      address = item['address']['addressLocality']

                      # 'aggregateRating' 유무 예외처리
                      aggregate_rating = item.get('aggregateRating', None)
                      if aggregate_rating:
                          rating_value = aggregate_rating['ratingValue']
                          review_count = aggregate_rating['reviewCount']
                      else:
                          rating_value = 'N/A'
                          review_count = 'N/A'

                      hotel_info['name'] = name
                      hotel_info['Url'] = url
                      hotel_info['star_rating'] = star_rating
                      hotel_info['price'] = price_range
                      hotel_info['Address'] = address
                      hotel_info['Rating'] = rating_value
                      hotel_info['review_cnt'] = review_count
                      print(name)
                      hotel_data.append(hotel_info)

             except Exception as e:
                      print(f"Error in Crawling page {page_num}: {e}")

    # 전체 dict를 JSON 파일로 저장
    with open('hotel_data.json', 'w', encoding='utf-8') as json_file:
      json.dump(hotel_data, json_file, ensure_ascii=False, indent=4)

    files.download('hotel_data.json')

    return 0

if __name__ == "__main__":
    region = input("지역명: ")
    start_date = input("체크인 날짜(0000-00-00): ")
    end_date = input("체크아웃 날짜(0000-00-00): ")
    person = input("인원수: ")
    total_page_num = get_total_page_num(region, start_date, end_date, person)
    print(f"총 검색 페이지 개수: {total_page_num}")
    if total_page_num > 0:
        Crawling(region, start_date, end_date, person, total_page_num)

지역명: 서울
체크인 날짜(0000-00-00): 2024-08-17
체크아웃 날짜(0000-00-00): 2024-08-18
인원수: 2
https://www.yeogi.com/domestic-accommodations?searchType=KEYWORD&keyword=서울&autoKeyword=&checkIn=2024-08-17&checkOut=2024-08-18&personal=2&freeForm=false&page=1
Connecting to Scraping Browser...
Connected! Navigating...
총 검색 호텔 개수: 1164
총 검색 페이지 개수: 59
Connecting to Scraping Browser...
Connected! Navigating...
서울신라호텔
길동 MARI-마리
스위스 그랜드 호텔
영등포 H-AVENUE
종로 K-World-종로 3가
사당 호텔 카일
신촌 라뉘호텔
종로 부티크 호텔K
신라스테이 구로
신촌 바론드 호텔
은평 씨에스에비뉴 호텔
이비스 스타일 앰배서더 서울 용산
영등포 Blvd 호텔오라
잠실 NU호텔-종합운동장점
신촌 가을
성신여대 호텔 우디
왕십리 포레스타
건대 더 디자이너스 프리미어-화양동
화곡 VOLL
★당일특가★ 앰배서더 서울 풀만 호텔
Connecting to Scraping Browser...
Connected! Navigating...
종로 브라운도트 호텔
역삼 리치웰
역삼 컬리넌
역삼 인트로호텔
강남 멜리샤호텔
성신여대역 더월-THE WALL
명동 밀리오레호텔
잠실 Stay hotel
강남 캠퍼스
신촌 호텔 루씨르
건대 호텔 K World
잠실 루이체
페어필드 바이 메리어트 서울
종로 시네마
종로 THE MAY HOTEL
독산 호텔 인 카페
영등포 더 문
신촌 림
잠실 라비호텔
사당 필름 37.2


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>